In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = trp
band = [4]
commitnumber = de5d43e
molecule = {'co2': 0.0004}
tsfc = 300


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o co2 band3b
  atmpro = mls
  band = 3b
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 0
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [5, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250)]
  tsfc = 294
  vmax = 720
  vmin = 620
  w_diffuse = [(1.66, 1.66, 1.66, 1.66, 1.66), (1.66, 1.66)]
  wgt = [(0, 0.6, 0.5, 0.7, 0.8), (0.8, 0.7)]


# Cooling Rate by Layer

CRD            CLIRAD `coolr_bands`           
            layer     coolrg                layer     coolrg
pressure                                                    
0.000312      1.0  76.911573                  NaN        NaN
0.000312      NaN        NaN                  1.0  38.962573
0.000750      2.0  41.020532                  2.0  33.983741
0.001052      3.0  33.259869                  3.0  31.877666
0.001476      4.0  26.397425                  4.0  28.876298
0.002070      5.0  20.664232                  5.0  24.961826
0.002904      6.0  16.081310                  6.0  20.318513
0.004074      7.0  12.530285                  7.0  15.386768
0.005714      8.0   9.827018                  8.0  10.841702
0.008015      9.0   7.770893                  9.0   7.334206
0.011243     10.0   6.180803                 10.0   5.171322
0.015771     11.0   4.903649                 11.0   4.086903
0.022122     12.0   3.810097                 12.0   3.487748
0.031031     13.0   2.745222                 13.0   2.860732
0.043528     14.0   1.399978                 14.0   1.768402
0.061057     15.0   0.979141                 15.0   1.368503
0.085645     16.0   1.752999                 16.0   1.972320
0.120136     17.0   2.555489                 17.0   2.532426
0.168516     18.0   3.176324                 18.0   2.874723
0.236378     19.0   3.831257                 19.0   3.362541
0.331549     20.0   4.628079                 20.0   4.277497
0.465100     21.0   5.780301                 21.0   5.775930
0.652400     22.0   7.472781                 22.0   7.732424
0.915100     23.0   7.603271                 23.0   7.836818
1.283650     24.0   6.451719                 24.0   6.295884
1.800600     25.0   5.474882                 25.0   5.050302
2.525700     26.0   4.427503                 26.0   4.245045
3.542800     27.0   3.636540                 27.0   3.880923
4.969550     28.0   3.059310                 28.0   3.333681
6.970850     29.0   2.591584                 29.0   2.635070
9.778100     30.0   2.179984                 30.0   2.144337
13.715850    31.0   1.799270                 31.0   2.004000
19.239350    32.0   1.443215                 32.0   1.764631
26.987250    33.0   1.121665                 33.0   1.294800
37.855300    34.0   0.874324                 34.0   0.889521
53.100050    35.0   0.507275                 35.0   0.567826
73.887500    36.0   0.034923                 36.0   0.130324
97.662500    37.0  -0.289156                 37.0  -0.278438
121.437500   38.0  -0.219236                 38.0  -0.255727
145.212500   39.0   0.020826                 39.0  -0.003524
168.987500   40.0   0.075532                 40.0   0.053535
192.762500   41.0   0.099899                 41.0   0.070742
216.537500   42.0   0.111780                 42.0   0.074970
240.312500   43.0   0.116942                 43.0   0.077449
264.087500   44.0   0.118045                 44.0   0.080924
287.862500   45.0   0.116553                 45.0   0.085225
311.637500   46.0   0.113290                 46.0   0.089377
335.412500   47.0   0.108849                 47.0   0.092626
359.187500   48.0   0.103653                 48.0   0.094000
382.962500   49.0   0.097977                 49.0   0.093271
406.737500   50.0   0.092141                 50.0   0.091018
430.512500   51.0   0.086304                 51.0   0.087214
454.287500   52.0   0.080623                 52.0   0.081834
478.062500   53.0   0.075138                 53.0   0.075202
501.837500   54.0   0.069950                 54.0   0.068797
525.612500   55.0   0.065174                 55.0   0.061919
549.387500   56.0   0.060815                 56.0   0.055555
573.162500   57.0   0.057038                 57.0   0.049710
596.937500   58.0   0.054224                 58.0   0.045023
620.712500   59.0   0.052987                 59.0   0.042299
644.487500   60.0   0.054949                 60.0   0.043148
668.262500   61.0   0.061355                 61.0   0.048814
692.037500   62.0   0.069008                 62.0

# Fluxes by Layer

CRD                           CLIRAD             \
                         flug       fldg      fnetg       flug       fldg   
pressure     level                                                          
0.000000e+00 1     -16.254068   0.000000 -16.254068        NaN        NaN   
1.000000e-08 1            NaN        NaN        NaN -15.668506   0.000183   
6.244000e-04 2     -16.254436   0.006056 -16.248379 -15.668891   0.003450   
8.759000e-04 3     -16.254603   0.007445 -16.247157 -15.669047   0.004619   
1.228600e-03 4     -16.254812   0.009044 -16.245768 -15.669250   0.006154   
1.723400e-03 5     -16.255074   0.010854 -16.244221 -15.669514   0.008110   
2.417400e-03 6     -16.255405   0.012883 -16.242522 -15.669857   0.010505   
3.390900e-03 7     -16.255824   0.015157 -16.240667 -15.670304   0.013296   
4.756500e-03 8     -16.256362   0.017721 -16.238640 -15.670890   0.016371   
6.672000e-03 9     -16.257057   0.020647 -16.236411 -15.671663   0.019605   
9.358900e-03 10    -16.257970   0.024033 -16.233937 -15.672693   0.022969   
1.312780e-02 11    -16.259189   0.028011 -16.231178 -15.674080   0.026665   
1.841450e-02 12    -16.260851   0.032745 -16.228107 -15.675988   0.031133   
2.583020e-02 13    -16.263185   0.038425 -16.224760 -15.678670   0.036878   
3.623230e-02 14    -16.266652   0.045275 -16.221377 -15.682600   0.044334   
5.082340e-02 15    -16.272641   0.053684 -16.218958 -15.689135   0.053926   
7.129060e-02 16    -16.282331   0.065747 -16.216584 -15.699370   0.067479   
1.000000e-01 17    -16.295514   0.084892 -16.210622 -15.713075   0.087893   
1.402710e-01 18    -16.312196   0.113765 -16.198431 -15.730221   0.117120   
1.967600e-01 19    -16.332902   0.155726 -16.177177 -15.751219   0.157357   
2.759970e-01 20    -16.357731   0.216515 -16.141216 -15.775985   0.213687   
3.871000e-01 21    -16.385269   0.304963 -16.080306 -15.802833   0.296837   
5.431000e-01 22    -16.408114   0.434624 -15.973490 -15.824140   0.424890   
7.617000e-01 23    -16.399970   0.619986 -15.779984 -15.815020   0.616020   
1.068500e+00 24    -16.348487   0.844825 -15.503661 -15.764851   0.850692   
1.498800e+00 25    -16.263914   1.089110 -15.174804 -15.683082   1.089869   
2.102400e+00 26    -16.142986   1.359641 -14.783346 -15.565385   1.333311   
2.949000e+00 27    -15.993169   1.653839 -14.339330 -15.418323   1.612010   
4.136600e+00 28    -15.814574   1.986831 -13.827743 -15.240446   1.980157   
5.802500e+00 29    -15.601602   2.377576 -13.224026 -15.026344   2.423983   
8.139200e+00 30    -15.348183   2.841504 -12.506679 -14.765636   2.892735   
1.141700e+01 31    -15.048074   3.387837 -11.660237 -14.449990   3.409775   
1.601470e+01 32    -14.695350   4.015049 -10.680300 -14.071644   4.122981   
2.246400e+01 33    -14.284245   4.706511  -9.577734 -13.626469   5.026064   
3.151050e+01 34    -13.800127   5.424394  -8.375733 -13.091339   5.878614   
4.420010e+01 35    -13.154004   6.092532  -7.061472 -12.358579   6.483096   
6.200000e+01 36    -12.436785   6.444915  -5.991870 -11.518771   6.840688   
8.577500e+01 37    -12.199267   6.305751  -5.893515 -11.209897   6.898886   
1.095500e+02 38    -12.862807   6.154936  -6.707871 -11.923883   6.828620   
1.333250e+02 39    -14.076385   6.751076  -7.325309 -13.249738   7.434191   
1.571000e+02 40    -15.303670   8.037015  -7.266656 -14.571609   8.746136   
1.808750e+02 41    -16.528801   9.474868  -7.053934 -15.874576  10.199888   
2.046500e+02 42    -17.745588  10.973001  -6.772587 -17.155109  11.679674   
2.284250e+02 43    -18.950082  12.492304  -6.457778 -18.415411  13.151135   
2.522000e+02 44    -20.139331  14.010899  -6.128432 -19.652129  14.605995   
2.759750e+02 45    -21.311691  15.515712  -5.795979 -20.866229  16.048028   
2.997500e+02 46    -22.466145  16.998415  -5.467729 -22.058434  17.480279   
3.235250e+02 47    -23.602153  18.453483  -5.148670 -23.226609  18.900193   
3.473000e+02 48    -24.719490  19.877374  -4.842116 -24.373177  20.307652   
3.710750e+02 49    -25

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')